## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-14-20-52-51 +0000,missionlocal,Vaquera/x/o on display: Michael Papias’ new ph...,https://missionlocal.org/2025/12/sf-michael-pa...
1,2025-12-14-20-52-26 +0000,nyt,Live Updates: 15 Victims and a Gunman Dead Aft...,https://www.nytimes.com/live/2025/12/14/world/...
2,2025-12-14-20-49-45 +0000,nypost,Trendy NYC area unplugs nose as asphalt plant ...,https://nypost.com/2025/12/14/us-news/trendy-n...
3,2025-12-14-20-48-00 +0000,wsj,Terrorist Attack Targets Jews at Sydney’s Bond...,https://www.wsj.com/world/oceania/sydney-bondi...
4,2025-12-14-20-48-00 +0000,wsj,Police Detain Person of Interest in Deadly Bro...,https://www.wsj.com/us-news/police-detain-pers...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2374/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
41,brown,24
34,bondi,21
43,shooting,19
35,beach,17
42,university,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
51,2025-12-14-18-41-53 +0000,nypost,President Trump mourns victims of Bondi Beach ...,https://nypost.com/2025/12/14/us-news/presiden...,133
6,2025-12-14-20-45-28 +0000,wapo,Live updates: Person of interest in Brown Univ...,https://www.washingtonpost.com/nation/2025/12/...,93
5,2025-12-14-20-46-31 +0000,wapo,Live updates: 15 killed in Bondi Beach shootin...,https://www.washingtonpost.com/world/2025/12/1...,87
69,2025-12-14-16-36-41 +0000,nypost,"Famed Australian rabbi Eli Schlanger, who just...",https://nypost.com/2025/12/14/world-news/famed...,84
4,2025-12-14-20-48-00 +0000,wsj,Police Detain Person of Interest in Deadly Bro...,https://www.wsj.com/us-news/police-detain-pers...,83


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
51,133,2025-12-14-18-41-53 +0000,nypost,President Trump mourns victims of Bondi Beach ...,https://nypost.com/2025/12/14/us-news/presiden...
69,46,2025-12-14-16-36-41 +0000,nypost,"Famed Australian rabbi Eli Schlanger, who just...",https://nypost.com/2025/12/14/world-news/famed...
6,33,2025-12-14-20-45-28 +0000,wapo,Live updates: Person of interest in Brown Univ...,https://www.washingtonpost.com/nation/2025/12/...
74,30,2025-12-14-15-50-19 +0000,nypost,Trump admits he’s not sure GOP will hold onto ...,https://nypost.com/2025/12/14/us-news/trump-ad...
122,26,2025-12-14-10-00-00 +0000,wsj,The falling out between Rep. Marjorie Taylor G...,https://www.wsj.com/politics/policy/marjorie-t...
87,24,2025-12-14-13-02-16 +0000,nypost,Incredible moment Bondi Beach hero disarms ter...,https://nypost.com/2025/12/14/world-news/incre...
25,21,2025-12-14-20-08-22 +0000,nypost,Chinese gaming billionaire reportedly sires mo...,https://nypost.com/2025/12/14/us-news/xu-bo-ch...
84,20,2025-12-14-13-50-33 +0000,nypost,Brown University instructor describes horror i...,https://nypost.com/2025/12/14/us-news/brown-un...
68,20,2025-12-14-17-00-00 +0000,wsj,"For decades, Tonka trucks have been a family s...",https://www.wsj.com/economy/trade/tonka-trucks...
16,19,2025-12-14-20-28-35 +0000,nyt,Zelensky Offers Compromise for New Round of Uk...,https://www.nytimes.com/2025/12/14/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
